In [1]:
import torch
import numpy as np

In [2]:
x = torch.rand(5, 3, dtype=torch.float32)
print(x)

tensor([[0.6232, 0.7119, 0.0774],
        [0.6153, 0.1724, 0.1612],
        [0.4381, 0.5741, 0.2611],
        [0.0500, 0.2592, 0.6033],
        [0.3952, 0.3935, 0.0655]])


In [3]:
# x = torch.tensor([5.5, 3])
print(x)

tensor([[0.6232, 0.7119, 0.0774],
        [0.6153, 0.1724, 0.1612],
        [0.4381, 0.5741, 0.2611],
        [0.0500, 0.2592, 0.6033],
        [0.3952, 0.3935, 0.0655]])


In [4]:
print(x.size())
print(x.shape)

torch.Size([5, 3])
torch.Size([5, 3])


In [5]:
result = torch.empty(5, 3, dtype=torch.float32)
print(result)

tensor([[0.0000, 0.0000, 0.0774],
        [0.6153, 0.1724, 0.1612],
        [0.4381, 0.5741, 0.2611],
        [0.0500, 0.2592, 0.6033],
        [0.3952, 0.3935, 0.0655]])


In [6]:
print(torch.add(x, result))

tensor([[0.6232, 0.7119, 0.1548],
        [1.2306, 0.3448, 0.3224],
        [0.8762, 1.1483, 0.5223],
        [0.1000, 0.5184, 1.2067],
        [0.7905, 0.7870, 0.1309]])


### view()改变维度
注意view()返回的新Tensor与源Tensor虽然可能有不同的size，但是是共享data的，也即更改其中的一个，另外一个也会跟着改变。(顾名思义，view仅仅是改变了对这个张量的观察角度，内部数据并未改变)

In [7]:
y = x.view(15)
z = x.view(-1, 5)  # -1所指的维度可以根据其他维度的值推出来
print(x.size(), y.size(), z.size())

torch.Size([5, 3]) torch.Size([15]) torch.Size([3, 5])


### clone()复制
如果我们想返回一个真正新的副本（即不共享data内存）该怎么办呢？Pytorch还提供了一个reshape()可以改变形状，但是此函数并不能保证返回的是其拷贝，所以不推荐使用。推荐先用clone创造一个副本然后再使用

In [8]:
x_cp = x.clone().view(15)
x -= 1
print(x)
print(x_cp)

tensor([[-0.3768, -0.2881, -0.9226],
        [-0.3847, -0.8276, -0.8388],
        [-0.5619, -0.4259, -0.7389],
        [-0.9500, -0.7408, -0.3967],
        [-0.6048, -0.6065, -0.9345]])
tensor([0.6232, 0.7119, 0.0774, 0.6153, 0.1724, 0.1612, 0.4381, 0.5741, 0.2611,
        0.0500, 0.2592, 0.6033, 0.3952, 0.3935, 0.0655])


### item()转换
将一个标量Tensor转换成一个Python number

In [9]:
x = torch.randn(1)
print(x)
print(x.item())

tensor([-0.3906])
-0.39062559604644775


### 广播机制 
当对两个形状不同的Tensor按元素运算时，可能会触发广播（broadcasting）机制：先适当复制元素使这两个Tensor形状相同后再按元素运算。
由于x和y分别是1行2列和3行1列的矩阵，如果要计算x + y，那么x中第一行的2个元素被广播（复制）到了第二行和第三行，而y中第一列的3个元素被广播（复制）到了第二列。如此，就可以对2个3行2列的矩阵按元素相加。

In [10]:
x = torch.arange(1, 3).view(1, 2)
print(x)
y = torch.arange(1, 4).view(3, 1)
print(y)
print(x + y)

tensor([[1, 2]])
tensor([[1],
        [2],
        [3]])
tensor([[2, 3],
        [3, 4],
        [4, 5]])


### Tensor和Numpy转换
    n = t.numpy()
    t = torch.from_numpy(n)共享内存/t = torch.tensor不共享内存

In [11]:
a = torch.ones(5)
b = a.numpy()
print(a, b)

tensor([1., 1., 1., 1., 1.]) [1. 1. 1. 1. 1.]


In [12]:
c = np.ones(5)
d = torch.from_numpy(c)
print(c, d)

[1. 1. 1. 1. 1.] tensor([1., 1., 1., 1., 1.], dtype=torch.float64)


### Tensor on GPU
用方法to()可以将Tensor在CPU和GPU（需要硬件支持）之间相互移动

In [13]:
if torch.cuda.is_available():
    device = torch.device("cuda")          # GPU
    y1 = torch.ones_like(x, device=device)  # 直接创建一个在GPU上的Tensor
    x1 = x.to(device)                       # 等价于 .to("cuda")
    z = x1 + y1
    print(z)
    print(z.to("cpu", torch.double))       # to()还可以同时更改数据类型

tensor([[2, 3]], device='cuda:0')
tensor([[2., 3.]], dtype=torch.float64)
